## loading dataset

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_proms = pd.read_csv('radoncPROMS.csv')


# 1. Select patients seen for breast condition, then drop empty columns

In [6]:
df_breast_proms = df_proms[df_proms['1. What_condition_is_this_patient_here_for?'] == 'Breast']
print len(df_breast_proms.index)

6261


In [13]:
df_breast = df_breast_proms.dropna(axis=1, how='all')
print df_breast.columns.values

['response_id' 'EMPI' 'MRN' 'FACT-G7_Score'
 'PROMIS-10_General_Physical_Health_Score'
 'PROMIS-10_General_Mental_Health_Score'
 '1. What_condition_is_this_patient_here_for?'
 '2. Is_the_patient_here_for_a_Baseline/Follow-up_appointment_or_OTV?'
 '10. Which_of_the_following_do_you_currently_have? (Own (native) bladder)'
 '10. Which_of_the_following_do_you_currently_have? (Ileal conduit/ ostomy)'
 '10. Which_of_the_following_do_you_currently_have? (Neo-bladder)'
 '10. Which_of_the_following_do_you_currently_have? (Continent urinary diversion/catheterizable pouch)'
 '10. Which_of_the_following_do_you_currently_have? (Other)'
 '70. In_the_past_7_days:_I_have_been_short_of_breath'
 '71. In_the_past_7_days:_I_am_self-conscious_about_the_way_I_dress'
 '72. In_the_past_7_days:_One_or_both_of_my_arms_are_swollen_or_tender'
 '73. In_the_past_7_days:I_feel_sexually_attractive'
 '74. In_the_past_7_days:_I_am_bothered_by_hair_loss'
 '75. In_the_past_7_days:_I_worry_that_other_members_of_my_family_

In [20]:
df_breast = df_breast.rename(columns={"1. What_condition_is_this_patient_here_for?":"condition", 
                          "2. Is_the_patient_here_for_a_Baseline/Follow-up_appointment_or_OTV?":'visit_type' })

## Checking how many visits of each type. 3240 baseline/fu visits and 3009 OTVs.

In [64]:
df_breast['visit_type'].value_counts(dropna=False)

Baseline/Follow-Up    3240
OTV                   3009
NaN                      2
Name: visit_type, dtype: int64

# 2. Drop entries missing both EMPI and MRN fields (10 rows). Then fill in patients missing EMPI with their MRN

In [62]:
# drop missing ids
df_missing_ids = df_breast[(df_breast['EMPI'] != df_breast['EMPI']) & (df_breast['MRN'] != df_breast['MRN'])]
df_breast = df_breast[(df_breast['EMPI'] == df_breast['EMPI']) | (df_breast['MRN'] == df_breast['MRN'])]
print len(df_breast.index)

6251


In [65]:
df_breast.loc[(df_breast['EMPI'] != df_breast['EMPI']), 'EMPI'] = df_breast['MRN']

# 3. Select OTVs only and group by identifier number

In [68]:
df_otvs = df_breast[df_breast['visit_type'] == 'OTV']
print len(df_otvs.index)

3009


In [69]:
print len(pd.unique(df_otvs['EMPI']))

1122


# 4. Get the distribution of how many OTVs a patient has. 
## 321 patients have 1 OTV
## 254 patients have 2 OTVs
## 249 patients have 3 OTVs
## 143 patients have 4 OTVs

In [75]:
print df_otvs.groupby('EMPI').size().value_counts()

1    321
2    254
3    249
4    143
5     89
6     49
7     14
8      3
dtype: int64


# 5. Creating a table with visits by date
## Need MRN to complete it

In [76]:
df_otvs_by_patient = pd.DataFrame(index=pd.unique(df_otvs['EMPI']), columns=['OTV1', 'OTV2', 'OTV3', 'OTV4', 'OTV5', 'OTV6', 'OTV7', 'OTV8'])
print len(df_otvs_by_patient.index)

1122


In [86]:
desired_length = len(df_otvs_by_patient.columns.values.tolist())
for empi in pd.unique(df_otvs['EMPI']):
    dates = df_otvs[df_otvs['EMPI'] == empi].loc[:,'date_of_submission']
    df_otvs_by_patient.loc[empi] = np.lib.pad(dates.values, (0, desired_length - dates.values.size), 'constant', constant_values=(0, np.nan))
    
print df_otvs_by_patient.head()
    

                    OTV1            OTV2            OTV3            OTV4  \
102226876  12/2/14 10:16    12/8/14 9:32  12/15/14 10:19   12/22/14 9:28   
109948880  12/2/14 16:01             NaN             NaN             NaN   
109959151   12/5/14 9:19             NaN             NaN             NaN   
107275392  12/5/14 11:14  12/12/14 10:58  12/19/14 11:14  12/24/14 11:50   
106873989  12/5/14 12:27  12/12/14 12:29  12/19/14 12:36             NaN   

                     OTV5 OTV6 OTV7 OTV8  
102226876             NaN  NaN  NaN  NaN  
109948880             NaN  NaN  NaN  NaN  
109959151             NaN  NaN  NaN  NaN  
107275392  12/30/14 11:20  NaN  NaN  NaN  
106873989             NaN  NaN  NaN  NaN  
